# Loading Responses

In [1]:
options(warn=1)
if(!require('lme4')) {
    install.packages('lme4', repos='https://cloud.r-project.org')
    library('lme4')
}
if(!require('readr')) {
    install.packages('readr', repos='https://cloud.r-project.org')
    library('readr')
}
if(!require('marginaleffects')) {
    install.packages('marginaleffects', repos='https://cloud.r-project.org')
    library('marginaleffects')
}
if(!require('emmeans')) {
    install.packages('emmeans', repos='https://cloud.r-project.org')
    library('emmeans')
}
if(!require('car')) {
    install.packages('car', repos='https://cloud.r-project.org')
    library('car')
}
df <- read_csv("gaze-analysis/result_1743457603_20250506_20250506F.csv", na = "empty", col_select = c("Accuracy", "Group", "GroupKind", "Angle", "Proximity", "n_candidates", "Actor", "Candidates", "Stimulus_ID", "Prompt_ID", "Participant_ID", "list_id", "Run_ID", "Part"), col_types = cols(
    Accuracy = col_logical(),
    Group = col_factor(),
    GroupKind = col_factor(),
    Angle = col_factor(),
    Proximity = col_integer(),
    n_candidates = col_integer(),
    Actor = col_factor(c('X', 'Y')),
    Candidates = col_factor(),
    Stimulus_ID = col_factor(),
    Prompt_ID = col_factor(),
    Participant_ID = col_factor(),
    list_id = col_factor(),
    Run_ID = col_character(),
    Part = col_character(),
),show_col_types = TRUE)
df <- df[df$Group == "Humans" & df$list_id != -1, ]
df$offset <- log(1/df$n_candidates / (1 - 1/df$n_candidates))

Loading required package: lme4

Loading required package: Matrix

Loading required package: readr

Loading required package: marginaleffects

Loading required package: emmeans

Welcome to emmeans.
Caution: You lose important information if you filter this package's results.
See '? untidy'

Loading required package: car

Loading required package: carData

Rows: 155393 Columns: 14
── Column specification ────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Run_ID, Part
int (2): Proximity, n_candidates
lgl (1): Accuracy
fct (9): Stimulus_ID, Prompt_ID, Participant_ID, Group, GroupKind, Angle, Ac...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# Statistics

In [2]:
mean(df[ (df$Proximity == 1) & (df$n_candidates == 2), ]$Accuracy)

[1] 0.9929701

In [18]:
length(unique(df$Participant_ID))

[1] 65

# Fitting models

In [3]:
model0 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Candidates) + (1|Stimulus_ID) + (1|Participant_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.241842 (tol = 0.002, component 1)”


In [4]:
model1 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Stimulus_ID) + (1|Participant_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model1

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Stimulus_ID) + (1 | Participant_ID)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
1312.7752 1360.6236 -648.3876 1296.7752      2917 
Random effects:
 Groups         Name        Std.Dev.
 Stimulus_ID    (Intercept) 1.6566  
 Participant_ID (Intercept) 0.6354  
Number of obs: 2925, groups:  Stimulus_ID, 900; Participant_ID, 65
Fixed Effects:
                       (Intercept)                           Angleleft  
                             7.414                              -3.236  
                        Angleright     scale(Proximity, scale = FALSE)  
                            -3.218                              -1.305  
scale(n_candidates, scale = FALSE)                              ActorY  
                  

In [5]:
model2 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

# Model selection

In [6]:
anova(model1, model2)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
model2,7,1326.876,1368.743,-656.4379,1312.876,NA,NA,NA
model1,8,1312.775,1360.624,-648.3876,1296.775,16.10054,1,6.006683e-05


In [7]:
model <- model1

# Interpretation

In [8]:
summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Stimulus_ID) + (1 | Participant_ID)
   Data: df
 Offset: offset

     AIC      BIC   logLik deviance df.resid 
  1312.8   1360.6   -648.4   1296.8     2917 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-7.6893  0.0358  0.0892  0.1932  2.0510 

Random effects:
 Groups         Name        Variance Std.Dev.
 Stimulus_ID    (Intercept) 2.7443   1.6566  
 Participant_ID (Intercept) 0.4037   0.6354  
Number of obs: 2925, groups:  Stimulus_ID, 900; Participant_ID, 65

Fixed effects:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                          7.4138     0.5270  14.068  < 2e-16 ***
Angleleft                           -3.2358     0.4217  -7.673 1.68e-14 ***
Angleright                       

In [9]:
vc_list <- lapply(VarCorr(model), function(x) attr(x, "stddev")^2)
vc <- sum(unlist(vc_list))
effect_size_denom_t <- sqrt(vc + pi^2/3)

res <- summary(model)$coefficients
res <- as.data.frame(res)
res <- cbind(term = rownames(res), res)
rownames(res) <- NULL
res$effect_size_denom <- effect_size_denom_t
res$effect_size <- res$Estimate / effect_size_denom_t
res

term,Estimate,Std. Error,z value,Pr(>|z|),effect_size_denom,effect_size
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),7.4138498,0.5270114,14.067722,5.996996e-45,2.537307,2.9219368
Angleleft,-3.2358288,0.4217099,-7.673115,1.678691e-14,2.537307,-1.2753006
Angleright,-3.2180958,0.4238400,-7.592713,3.132747e-14,2.537307,-1.2683117
"scale(Proximity, scale = FALSE)",-1.3047983,0.1677296,-7.779178,7.299743e-15,2.537307,-0.5142454
"scale(n_candidates, scale = FALSE)",-0.6700431,0.1518226,-4.413329,1.017931e-05,2.537307,-0.2640765
ActorY,-1.0032615,0.2383074,-4.209947,2.554306e-05,2.537307,-0.3954041


In [10]:
avg_predictions(model, by="Angle", hypothesis = ~ pairwise)

hypothesis,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(left) - (front),-0.079567747,0.01356133,-5.8672512,4.430791e-09,27.7497885,-0.10614747,-0.05298802
(right) - (front),-0.074370462,0.01272584,-5.8440532,5.094575e-09,27.5483911,-0.09931264,-0.04942828
(right) - (left),0.005197285,0.01418944,0.3662782,7.141575e-01,0.4856859,-0.02261352,0.03300809


In [11]:
avg_slopes(model, variables ="Proximity", type = "link")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Proximity,dY/dX,-1.304798,0.167737,-7.778833,7.319637e-15,46.95715,-1.633557,-0.9760398,7.251832,7.251572,7.251702


In [12]:
avg_slopes(model, variables ="n_candidates", type = "link")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
n_candidates,dY/dX,-0.6700431,0.1517818,-4.414516,1.012361e-05,16.59192,-0.9675299,-0.3725563,7.251769,7.251635,7.251702


In [13]:
avg_predictions(model, by="Actor", hypothesis = ~ pairwise)

hypothesis,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Y) - (X),-0.04517929,0.01032169,-4.377123,1.202562e-05,16.34353,-0.06540943,-0.02494916
